# Asynchronous Advantage Actor-Critic (A3C) with gradients parallelism

In [1]:
import sys
sys.path.append("../Chapter13/")

In [2]:
import os
import gym
import ptan
import argparse
from tensorboardX import SummaryWriter

import torch
import torch.nn.utils as nn_utils
import torch.nn.functional as F
import torch.optim as optim
import torch.multiprocessing as mp

from lib import common

As usual, we define the hyperparameters, which are mostly the same as in the
previous example, except BATCH_SIZE is replaced by two parameters: 
- GRAD_BATCH
- TRAIN_BATCH. 

The value of GRAD_BATCH defines the size of the batch used by
every child process to compute the loss and get the value of the gradients. The
second parameter, TRAIN_BATCH, specifies how many gradient batches from the child
processes will be combined on every SGD iteration. Every entry produced by the
child process has the same shape as our network parameters, and we sum up TRAIN_
BATCH values of them together. So, for every optimization step, we use the TRAIN_
BATCH * GRAD_BATCH training samples. As the loss calculation and backpropagation
are quite heavy operations, we use large GRAD_BATCH to make them more efficient.

Due to this large batch, we should keep TRAIN_BATCH relatively low to keep our
network update on-policy.

In [3]:
GAMMA = 0.99
LEARNING_RATE = 0.001
ENTROPY_BETA = 0.01

REWARD_STEPS = 4
CLIP_GRAD = 0.1

PROCESSES_COUNT = 4
NUM_ENVS = 8

GRAD_BATCH = 64
TRAIN_BATCH = 2

In [4]:
if True:
    ENV_NAME = "PongNoFrameskip-v4"
    NAME = 'pong'
    REWARD_BOUND = 18
else:
    ENV_NAME = "BreakoutNoFrameskip-v4"
    NAME = "breakout"
    REWARD_BOUND = 400
    TRAIN_BATCH = 4


def make_env():
    return ptan.common.wrappers.wrap_dqn(gym.make(ENV_NAME))


def grads_func(proc_name, net, device, train_queue):
    envs = [make_env() for _ in range(NUM_ENVS)]

    agent = ptan.agent.PolicyAgent(
        lambda x: net(x)[0], device=device, apply_softmax=True)
    exp_source = ptan.experience.ExperienceSourceFirstLast(
        envs, agent, gamma=GAMMA, steps_count=REWARD_STEPS)

    batch = []
    frame_idx = 0
    writer = SummaryWriter(comment=proc_name)

    with common.RewardTracker(writer, REWARD_BOUND) as tracker:
        with ptan.common.utils.TBMeanTracker(
                writer, 100) as tb_tracker:
            for exp in exp_source:
                frame_idx += 1
                new_rewards = exp_source.pop_total_rewards()
                if new_rewards and tracker.reward(
                        new_rewards[0], frame_idx):
                    break

                batch.append(exp)
                if len(batch) < GRAD_BATCH:
                    continue

                data = common.unpack_batch(
                    batch, net, device=device,
                    last_val_gamma=GAMMA**REWARD_STEPS)
                states_v, actions_t, vals_ref_v = data

                batch.clear()

                net.zero_grad()
                logits_v, value_v = net(states_v)
                loss_value_v = F.mse_loss(
                    value_v.squeeze(-1), vals_ref_v)

                log_prob_v = F.log_softmax(logits_v, dim=1)
                adv_v = vals_ref_v - value_v.detach()
                log_p_a = log_prob_v[range(GRAD_BATCH), actions_t]
                log_prob_actions_v = adv_v * log_p_a
                loss_policy_v = -log_prob_actions_v.mean()

                prob_v = F.softmax(logits_v, dim=1)
                ent = (prob_v * log_prob_v).sum(dim=1).mean()
                entropy_loss_v = ENTROPY_BETA * ent

                loss_v = entropy_loss_v + loss_value_v + \
                         loss_policy_v
                loss_v.backward()

                tb_tracker.track("advantage", adv_v, frame_idx)
                tb_tracker.track("values", value_v, frame_idx)
                tb_tracker.track("batch_rewards", vals_ref_v,
                                 frame_idx)
                tb_tracker.track("loss_entropy", entropy_loss_v,
                                 frame_idx)
                tb_tracker.track("loss_policy", loss_policy_v,
                                 frame_idx)
                tb_tracker.track("loss_value", loss_value_v,
                                 frame_idx)
                tb_tracker.track("loss_total", loss_v, frame_idx)

                # gather gradients
                nn_utils.clip_grad_norm_(
                    net.parameters(), CLIP_GRAD)
                grads = [
                    param.grad.data.cpu().numpy()
                    if param.grad is not None else None
                    for param in net.parameters()
                ]
                train_queue.put(grads)

    train_queue.put(None)

In [5]:
mp.set_start_method('spawn')
os.environ['OMP_NUM_THREADS'] = "1"

device = "cuda"

env = make_env()
net = common.AtariA2C(env.observation_space.shape,
                      env.action_space.n).to(device)
net.share_memory()

optimizer = optim.Adam(net.parameters(),
                       lr=LEARNING_RATE, eps=1e-3)

train_queue = mp.Queue(maxsize=PROCESSES_COUNT)
data_proc_list = []
for proc_idx in range(PROCESSES_COUNT):
    proc_name = f"-a3c-grad_pong_test_ac3_grad"
    p_args = (proc_name, net, device, train_queue)
    data_proc = mp.Process(target=grads_func, args=p_args)
    data_proc.start()
    data_proc_list.append(data_proc)

batch = []
step_idx = 0
grad_buffer = None

try:
    while True:
        train_entry = train_queue.get()
        if train_entry is None:
            break

        step_idx += 1

        if grad_buffer is None:
            grad_buffer = train_entry
        else:
            for tgt_grad, grad in zip(grad_buffer,
                                      train_entry):
                tgt_grad += grad

        if step_idx % TRAIN_BATCH == 0:
            for param, grad in zip(net.parameters(),
                                   grad_buffer):
                param.grad = torch.FloatTensor(grad).to(device)

            nn_utils.clip_grad_norm_(
                net.parameters(), CLIP_GRAD)
            optimizer.step()
            grad_buffer = None
finally:
    for p in data_proc_list:
        p.terminate()
        p.join()

KeyboardInterrupt: 